<a href="https://colab.research.google.com/github/plue1011/GNN/blob/master/pfn2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題1

In [0]:
import numpy as np

In [0]:
def ReLU(x):
    return np.maximum(0, x)

# ReLU(np.array([1,-1,3]))
# array([1, 0, 3])

In [0]:
class Graph:
    def __init__(self, _A):
        self.A = _A  # 隣接行列
        self.V = self.A.shape[0]  # 頂点数
        self.T = 2  # 集約回数

    # 集約関数
    def Aggregate(self, W):
        D = W.shape[0]  # パラメータの次元
        F = np.zeros((D, self.V))  # 各頂点の特徴ベクトルを横に並べたD☓Vのゼロ行列
        F[0] = 1  # 各頂点の特徴ベクトルの最初の要素を1に初期化

        for i in range(self.T):
            F = ReLU(np.dot(W, np.dot(F, self.A)))

        return np.dot(F, np.ones((self.V, 1)))

In [29]:
A = np.array([[0, 1, 0, 0],
              [1, 0, 1, 1],
              [0, 1, 0, 1],
              [0, 1, 1, 0]])
G = Graph(A)
W1 = np.ones((8, 8))
W2 = np.triu(np.ones((8, 8)))
Vec1 = G.Aggregate(W1)
Vec2 = G.Aggregate(W2)
F0 = np.zeros((8, 4))
F0[0] = 1

print("test")
print(Vec1)
print(((W1.dot(W1)).dot(F0)).dot((A.T).dot(A.T)).dot(np.ones((4, 1))))
print(Vec2)
print(((W2.dot(W2)).dot(F0)).dot((A.T).dot(A.T)).dot(np.ones((4, 1))))

[[24. 40. 40. 40.]
 [24. 40. 40. 40.]
 [24. 40. 40. 40.]
 [24. 40. 40. 40.]
 [24. 40. 40. 40.]
 [24. 40. 40. 40.]
 [24. 40. 40. 40.]
 [24. 40. 40. 40.]] [[1.]
 [1.]
 [1.]
 [1.]]
[[3. 5. 5. 5.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] [[1.]
 [1.]
 [1.]
 [1.]]
test
[[144.]
 [144.]
 [144.]
 [144.]
 [144.]
 [144.]
 [144.]
 [144.]]
[[144.]
 [144.]
 [144.]
 [144.]
 [144.]
 [144.]
 [144.]
 [144.]]
[[18.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[[18.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]


In [17]:
np.ones((1,3))

array([[1., 1., 1.]])